# day 11

https://adventofcode.com/11/day/11

In [ ]:
import logging
import logging.config
import os

import networkx as nx
import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day11.txt')

LOGGER = logging.getLogger('day11')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """aaa: you hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read()

In [ ]:
import networkx as nx

def parse_raw_data(data: str) -> nx.DiGraph:
    d = {row.split(': ')[0]: row.split(': ')[1].split(' ') for row in data.strip().split('\n')}

    g = nx.DiGraph()
    g.add_nodes_from(d.keys())
    g.add_edges_from((k, v) for (k, v_list) in d.items() for v in v_list)
    return g

In [ ]:
g = parse_raw_data(data=test_data)
g.edges(data=True)

In [ ]:
list(nx.all_simple_paths(g, 'you', 'out'))

#### function def

In [ ]:
def q_1(data):
    g = parse_raw_data(data=data)
    return sum(1 for _ in nx.all_simple_paths(g, 'you', 'out'))

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 5
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def num_paths(g: nx.DiGraph, start: str, end: str) -> int:
    return sum(1 for _ in nx.all_simple_paths(g, start, end, cutoff=10))


def q_2(data):
    g = parse_raw_data(data=data)

    d = {k: nx.shortest_path_length(g, 'svr', k) for k in ['dac', 'fft']}
    [a, b] = sorted(d.keys(), key=lambda n: d[n])

    mid_layer_depth = sum(d.values()) // 2

    nodes_at_mid_layer = nx.descendants_at_distance(g, 'svr', mid_layer_depth)

    LOGGER.warning("leg1")
    n_leg1 = num_paths(g=g, start='svr', end=a)

    LOGGER.warning("leg2")
    n_leg2 = sum([
        num_paths(g, start=a, end=midnode) * (num_paths(g, start=midnode, end=b))
        for midnode in nodes_at_mid_layer
    ])

    LOGGER.warning("leg3")
    n_leg3 = num_paths(g=g, start=b, end='out')

    return n_leg1 * n_leg2 * n_leg3

q_2(load_data())

In [ ]:
import pandas as pd

df = pd.DataFrame({'betweenness': nx.betweenness_centrality(g),
                   'degree': dict(g.degree())})
df.sort_values(by='betweenness', ascending=False, inplace=True)
df

In [ ]:
[(i, len(l), 'fft' if 'fft' in l else ('dac' if 'dac' in l else None)) for (i, l) in enumerate(nx.bfs_layers(g, 'svr'))]

In [ ]:
sum(1 for _ in nx.all_simple_paths(g, 'svr', 'fft', cutoff=10))

In [ ]:
for i in range(20):
    z = sum(1 for _ in nx.all_simple_paths(g, 'fft', 'dac', cutoff=i))
    print(f"{i = }, {z = }")

#### tests

In [ ]:
test_data = """svr: aaa bbb
aaa: fft
fft: ccc
bbb: tty
tty: ccc
ccc: ddd eee
ddd: hub
hub: fff
eee: dac
dac: fff
fff: ggg hhh
ggg: out
hhh: out
"""

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 2
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

In [ ]:
g = parse_raw_data(data=load_data())

In [ ]:
g.size()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(30, 30))
nx.draw(g, ax=ax, node_size=100, with_labels=True)
# nx.draw_random(g, ax=ax)
# nx.draw_circular(g, ax=ax)
# nx.draw_spectral(g, ax=ax)
# nx.draw_spring(g, ax=ax)

In [ ]:
# import pandas as pd
#
# df = pd.DataFrame({'betweenness': nx.betweenness_centrality(g),
#                    'degree': dict(g.degree())})
# df.sort_values(by='degree', ascending=False, inplace=True)
# df.head(50)

In [ ]:
nx.shortest_path_length(g, 'svr', 'dac')

In [ ]:
nx.shortest_path_length(g, 'svr', 'fft')

In [ ]:
q_2(load_data())

fin

In [ ]:
# from online
# INPUT_FILE = "input.txt"
INPUT_FILE = FNAME

graph = {}
with open(INPUT_FILE, "r") as f:
    for devices in [line.strip().split() for line in f]:
        graph[devices[0][:-1]] = devices[1:]

def traverse(device, end, visited, scores):
    if device == end:
        return 1
    if device in visited or device == "out":
        return 0
    if device in scores:
        return scores[device]
    visited.add(device)
    total = sum([traverse(output, end, visited, scores) for output in graph[device]])
    visited.remove(device)
    scores[device] = total
    return total

# Part1
print(traverse("you", "out", set(), {}))

# Part2
a1 = traverse("svr", "fft", set(), {})
a2 = traverse("fft", "dac", set(), {})
a3 = traverse("dac", "out", set(), {})
b1 = traverse("svr", "dac", set(), {})
b2 = traverse("dac", "fft", set(), {})
b3 = traverse("fft", "out", set(), {})
print(a1*a2*a3 + b1*b2*b3)
